From the main directory "Ontolearn", run the commands for NCES data in [`./download_external_resources.sh`](../download_external_resources.sh) to download pretrained models and datasets.

## Inference with NCES

In [1]:
from ontolearn.concept_learner import NCES
from ontolearn.knowledge_base import KnowledgeBase
from ontolearn.owlapy.parser import DLSyntaxParser

In [2]:
nces = NCES(knowledge_base_path="../NCESData/family/family.owl", learner_name="SetTransformer",
     path_of_embeddings="../NCESData/family/embeddings/ConEx_entity_embeddings.csv", load_pretrained=True, max_length=48, proj_dim=128, rnn_n_layers=2, drop_prob=0.1, num_heads=4, num_seeds=1, num_inds=32, pretrained_model_name="SetTransformer")



 Loaded pretrained model! 



In [3]:
KB = KnowledgeBase(path=nces.knowledge_base_path)

In [4]:
#list(KB.ontology().classes_in_signature())

In [5]:
dl_parser = DLSyntaxParser(nces.kb_namespace)

### Let's learn the concept ``Father''

In [6]:
father = dl_parser.parse('Father')

In [7]:
not_father = dl_parser.parse('¬Father') # For negative examples

In [8]:
pos = [ind.get_iri().as_str().split("/")[-1] for ind in KB.individuals(father)]

In [9]:
neg = [ind.get_iri().as_str().split("/")[-1] for ind in KB.individuals(not_father)]

#### Prediction with SetTransformer

In [10]:
concept = nces.fit(pos, neg)

Prediction:  Father


In [11]:
nces.sorted_examples

True

#### Prediction with GRU

In [12]:
nces.pretrained_model_name = 'GRU'
nces.refresh()
concept = nces.fit(pos, neg)



 Loaded pretrained model! 

Prediction:  Father


#### Prediction with LSTM

In [13]:
nces.pretrained_model_name = 'LSTM'
nces.refresh()
concept = nces.fit(pos, neg)



 Loaded pretrained model! 

Prediction:  Father ⊔ (∃ hasChild.(Daughter ⊓ (∃ hasChild.Grandfather)))


#### Prediction with ensemble SetTransformer+GRU+LSTM

In [14]:
nces.pretrained_model_name = ['SetTransformer','GRU','LSTM']
nces.refresh()
concept = nces.fit(pos, neg)



 Loaded pretrained model! 



 Loaded pretrained model! 



 Loaded pretrained model! 

Prediction:  Father


### Scalability of NCES (solving multiple learning problems in a go!)

In [15]:
import json, time

In [16]:
with open("../NCESData/family/training_data/Data.json") as file:
    data = list(json.load(file).items())
## The function below takes an iterable of tuples
LPs = list(map(lambda x: (x[0], x[1]["positive examples"], x[1]["negative examples"]), data))

In [17]:
## We solve 256 learning problems!
lps = LPs[:256]

In [18]:
t0 = time.time()
concepts = nces.fit_from_iterable(lps, verbose=False) ## Now predict with nces.fit_from_iterable.
t1 = time.time()
print("Duration: ", t1-t0, "seconds!")

Duration:  3.079460620880127 seconds!


In [19]:
print(concepts[0])

OWLClass(IRI('http://www.benchmark.org/family#','Sister'))


In [20]:
print(len(concepts))

256


### Change pretrained model name, e.g., use ensemble model prediction

In [21]:
nces.pretrained_model_name = ['SetTransformer', 'GRU']

In [22]:
nces.pretrained_model_name

['SetTransformer', 'GRU']

In [23]:
nces.refresh()



 Loaded pretrained model! 



 Loaded pretrained model! 



In [24]:
t0 = time.time()
concepts = nces.fit_from_iterable(lps, verbose=False)
t1 = time.time()
print("Duration: ", t1-t0, "seconds!")

Duration:  8.768116474151611 seconds!


In [25]:
concepts[-1]

OWLObjectAllValuesFrom(property=OWLObjectProperty(IRI('http://www.benchmark.org/family#','married')),filler=OWLObjectComplementOf(OWLClass(IRI('http://www.benchmark.org/family#','PersonWithASibling'))))

## Training NCES

In [26]:
import json, time
with open("../NCESData/family/training_data/Data.json") as file:
    data = json.load(file) # Training data. Below we use the first 200 datapoints to train the synthesizer. You train on the full data, e.g. on GPU by remobing "[:200]"

In [27]:
from ontolearn.concept_learner import NCES

In [28]:
nces = NCES(knowledge_base_path="../NCESData/family/family.owl", learner_name="SetTransformer",
     path_of_embeddings="../NCESData/family/embeddings/ConEx_entity_embeddings.csv", max_length=48, proj_dim=128, rnn_n_layers=2, drop_prob=0.1, num_heads=4, num_seeds=1, num_inds=32,
            load_pretrained=False, pretrained_model_name="SetTransformer")

In [29]:
nces.train(list(data.items())[-200:], epochs=20, learning_rate=0.001, save_model=True, storage_path=f"./NCES-{time.time()}/")

********************Trainable model size********************
Synthesizer:  515296
********************Trainable model size********************

Training on CPU, it may take long...

##################################################

SetTransformer starts training... 

################################################## 



Loss: 0.5712, Soft Acc: 74.01%, Hard Acc: 71.02%: 100%|██████████| 20/20 [01:12<00:00,  3.63s/it]

Top performance: loss: 0.5712, soft accuracy: 74.65% ... hard accuracy: 72.23%

SetTransformer saved
